To prepare: 
1. Pancreas - 4nm
2. HeLa 2&3 - 4nm
3. Fly fan shaped body(?) - 4nm; we don’t have this
4. Macrophage - 4nm
5. Jurkat - 4nm
6. SUM159 - 4nm
7. TCell on cancer - 4nm; I don’t know if we have the most recent dataset
8. HeLa mitotic - 4nm; we don’t have this


In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from fst.scripts.python.consolidate_dataset import prepare_store, dataset, name_prediction_array, DataArrayFactory
import dask
import dask.array as da
from tqdm import tqdm

In [ ]:
sources = {}
fibsem_path = Path('/groups/cosem/cosem/data/HeLa_Cell2_4x4x4nm/HeLa_Cell2_4x4x4nm.n5/volumes/raw/')
sources['fibsem'] = DataArrayFactory(fibsem_path, dest_path='fibsem', chunks='auto')

preds = [Path('/nrs/cosem/cosem/training/v0003.2/setup03/HeLa_Cell2_4x4x4nm/HeLa_Cell2_4x4x4nm_it625000.n5/er/'),
         Path('/nrs/cosem/cosem/training/v0003.2/setup03/HeLa_Cell2_4x4x4nm/HeLa_Cell2_4x4x4nm_it625000.n5/plasma_membrane/'),
         Path('/nrs/cosem/cosem/training/v0003.2/setup25/HeLa_Cell2_4x4x4nm/HeLa_Cell2_4x4x4nm_it625000.n5/mito/'),
         Path('/nrs/cosem/cosem/training/v0003.2/setup03/HeLa_Cell2_4x4x4nm/HeLa_Cell2_4x4x4nm_it625000.n5/nucleus/')]

for pred in preds:
    path = 'prediction/' + name_prediction_array(pred)
    array = DataArrayFactory(pred, path, 'auto')
    sources[path] = array
    
ds = dataset(name='jrc_hela_a01', sources=sources)

In [ ]:
sources = {}
fibsem_path = Path('/groups/cosem/cosem/data/Macrophage_FS80_Cell2_4x4x4nm/Cryo_FS80_Cell2_4x4x4nm.n5/volumes/raw/')
sources['fibsem'] = DataArrayFactory(fibsem_path, dest_path='fibsem', chunks='auto')

preds = [Path('/nrs/cosem/cosem/training/v0003.2/setup03/Macrophage_FS80_Cell2_4x4x4nm/Cryo_FS80_Cell2_4x4x4nm_it650000.n5/er/'),
         Path('/nrs/cosem/cosem/training/v0003.2/setup03/Macrophage_FS80_Cell2_4x4x4nm/Cryo_FS80_Cell2_4x4x4nm_it650000.n5/plasma_membrane/'),
         Path('/nrs/cosem/cosem/training/v0003.2/setup03/Macrophage_FS80_Cell2_4x4x4nm/Cryo_FS80_Cell2_4x4x4nm_it500000.n5/mito/'),
         Path('/nrs/cosem/cosem/training/v0003.2/setup03/Macrophage_FS80_Cell2_4x4x4nm/Cryo_FS80_Cell2_4x4x4nm_it625000.n5/nucleus/')]

for pred in preds:
    path = 'prediction/' + name_prediction_array(pred)
    array = DataArrayFactory(pred, path, 'auto')
    sources[path] = array
    
ds = dataset(name='macrophage', sources=sources)

In [ ]:
sources = {}
fibsem_path = Path('/groups/cosem/cosem/data/jrc_ctl-id8_a00/jrc_ctl-id8_a00.n5/volumes/aligned/')
sources['fibsem'] = DataArrayFactory(fibsem_path, dest_path='fibsem', chunks='auto')
ds = dataset(name='jrc_ctl-id8_a00', sources=sources)

In [ ]:
sources = {}
fibsem_path = Path('/groups/cosem/cosem/bennettd/drosophila_FSB_Col_1-9_Z0519-11_4x4x4nm/drosophila_FSB_Col_1-9_Z0519-11_4x4x4nm.n5/volumes/raw')
sources['fibsem'] = DataArrayFactory(fibsem_path, dest_path='fibsem', chunks='auto')
ds = dataset(name='jrc_fly-fsb_a00', sources=sources)

In [ ]:
output_dir = '/nrs/cosem/davis/s3_testing/'
to_store = prepare_store(ds, output_dir, {'z': 2, 'y': 2, 'x': 2})

In [ ]:
from fst.distributed import get_cluster
cl = get_cluster(walltime='300')
cl

In [ ]:
cl.cluster.scale(150)
results = []
for pyr in tqdm(to_store):
    results.append(cl.gather(cl.compute(pyr)))
cl.cluster.scale(0)